In [ ]:
# LIBRARIES
# %matplotlib inline

# system libraries

import os
from glob import glob
from tqdm import tqdm
import csv

# numpy and pandas
import pandas as pd
import numpy as np
from numpy import save
from numpy import load

# matplotlib and visualization libs
import cv2
import matplotlib.pyplot as plt
from skimage.transform import resize
import seaborn as sns
import itertools
from PIL import Image

# sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.model_selection import KFold

# tensorflow
import tensorflow as tf
from tensorflow import keras
from tensorflow import config
from tensorflow.keras.models import model_from_json
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, EarlyStopping 

## AI

In [ ]:

np.random.seed(123)

print("Num GPUs Available: ", len( config.experimental.list_physical_devices('GPU')))
##physical_devices = config.list_physical_devices("GPU")

#config.experimental.set_memory_growth(physical_devices[0], True)

physical_devices = config.experimental.list_physical_devices('GPU')
if physical_devices:
  try:
    for physical_device in physical_devices:
      config.experimental.set_memory_growth(physical_device, True)
  except RuntimeError as e:
    print(e)
    
tf.config.set_visible_devices(physical_devices[2], 'GPU')

#%%  
def plot_confusion_matrix(cm, classes,
                          model_no=1,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.savefig(f"../models/plots/kfold_{model_no}.png")
    plt.show()

In [ ]:
from sklearn.preprocessing import LabelEncoder

# %% variables
    
classes = {"normal": 0,
           "implant": 1,
           "fracture": 2,
           }


batch_size = 8
loss_function = "categorical_crossentropy"
input_shape = (299, 299, 3)
num_classes = 3
verbosity = 1
num_folds = 5
# %%
#model = load_model('/content/drive/MyDrive/Skin Cancer/Data_models/model.h5')

x_data = load("DATA_PATH")
y_data = load("DATA_PATH")

print("0:" + str(np.count_nonzero(y_data==0)))
print("1:" + str(np.count_nonzero(y_data==1)))
print("2:" + str(np.count_nonzero(y_data==2)))

# %%
X_train, X_test, y_train, y_test = train_test_split(x_data, y_data, test_size = 0.15, random_state = 28,stratify = y_data)
X_train, X_test, y_train, y_test = train_test_split(x_data, y_data, test_size = 0.1, random_state = 28, stratify = y_data)

root_dir = "ROOT_DIR"  # Replace with your root directory
img_size = (299, 299)

X_data = []
y_data = []
class_names = sorted(os.listdir(root_dir))  # Get class names in sorted order

for label, class_name in enumerate(class_names):
    class_dir = os.path.join(root_dir, class_name)
    
    if not os.path.isdir(class_dir):
        continue  # Skip files, only process directories
    
    print(f"Processing class: {class_name}")

    for file_name in tqdm(os.listdir(class_dir)):
        file_path = os.path.join(class_dir, file_name)
        try:
            img = cv2.imread(file_path)
            if img is None:
                continue  # Skip unreadable images
            
            img = cv2.resize(img, img_size)  # Resize image
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert to RGB
            X_data.append(img)
            y_data.append(class_name)  # Store class name instead of label index

        except Exception as e:
            print(f"Error reading {file_path}: {e}")

X_data = np.array(X_data, dtype=np.uint8)  # Convert list to numpy array
y_data = np.array(y_data)

# Encode labels as integers
label_encoder = LabelEncoder()
y_data = label_encoder.fit_transform(y_data)

print("Dataset loaded successfully!")
print(f"Total images: {len(X_data)}")
print(f"Class names: {class_names}")

In [ ]:
X_train = X_train.reshape(X_train.shape[0], *(299, 299, 3))
X_val = X_val.reshape(X_val.shape[0], *(299, 299, 3))
X_test = X_test.reshape(X_test.shape[0], *(299, 299, 3))

# Parse numbers as floats
X_train = X_train.astype('float32')
X_val = X_val.astype('float32')
X_test = X_test.astype('float32')


y_train = to_categorical(y_train, num_classes=num_classes)
y_val = to_categorical(y_val, num_classes=num_classes)
y_test = to_categorical(y_test, num_classes=num_classes)


# %%
train_datagen = ImageDataGenerator(
    featurewise_center=False,  # set input mean to 0 over the dataset
    samplewise_center=False,  # set each sample mean to 0
    featurewise_std_normalization=False,  # divide inputs by std of the dataset
    samplewise_std_normalization=False,  # divide each input by its std
    zca_whitening=False,  # apply ZCA whitening
    # randomly rotate images in the range (degrees, 0 to 180)
    rotation_range=45,
    zoom_range=0.2,  # Randomly zoom image
    # randomly shift images horizontally (fraction of total width)
    width_shift_range=0.2,
    # randomly shift images vertically (fraction of total height)
    height_shift_range=0.2,
    horizontal_flip=True,  # randomly flip images
)

train_datagen.fit(X_train)

optimizer = Adam(learning_rate=0.0001, beta_1=0.9, beta_2=0.999, decay=0.0, amsgrad=False)

#%%

# Define the K-fold Cross Validator
kfold = KFold(n_splits = num_folds, shuffle=True)

In [ ]:
from tensorflow.keras import mixed_precision

# Enable mixed precision
mixed_precision.set_global_policy('mixed_float16')

In [ ]:
model_no = 1

base_model = keras.applications.Xception(
    weights='imagenet',
    input_shape=input_shape,
    include_top=False
)

## callbacks part
mc_path = f"../models/best_models/{base_model.name}_kfold-model{model_no}.weights.h5"


model_checkpoint = ModelCheckpoint(
    filepath = mc_path,
    monitor = 'val_accuracy',
    mode = 'max',
    verbose = 1,
    save_best_only = True,
    save_weights_only = True
)

learning_rate_reduction = ReduceLROnPlateau(
    monitor='val_accuracy',
    patience=2,
    verbose=1,
    factor=0.1,
    min_lr=0.00001
)

log_csv = CSVLogger(f'../models/model_logs/{base_model.name}_model{model_no}_logs.csv', separator = ',', append = False)
early_stopping = EarlyStopping(monitor='loss', patience = 10)

callbacks_list = [learning_rate_reduction, model_checkpoint, log_csv]


base_modelQ4 = base_model

base_modelQ4.trainable = False
inputs = keras.Input(shape=input_shape)


q4 = base_modelQ4(inputs, training=False)
q4 = keras.layers.Conv2D(64, (1,1), activation = 'relu', padding = 'same', name = "pikachu")(q4)
q4 = keras.layers.GlobalAveragePooling2D()(q4)
q4 = keras.layers.Dropout(0.2)(q4)
q4 = keras.layers.Flatten()(q4)
q4 = keras.layers.Dense(256, activation='relu')(q4)
q4 = keras.layers.Dropout(0.2)(q4)
q4 = keras.layers.Dense(128, activation='relu')(q4)
q4 = keras.layers.Dropout(0.3)(q4)
q4 = keras.layers.Dense(64, activation='relu')(q4)
q4 = keras.layers.Dropout(0.3)(q4)


outputs = keras.layers.Dense(num_classes, activation='softmax')(q4)
modelQ4 = keras.Model(inputs, outputs)

modelQ4.compile(
    loss= loss_function,
    metrics=["accuracy"],
                optimizer=optimizer)

print(y_train.shape)

epochs = 50
modelQ4.fit(
    train_datagen.flow(
        X_train[train], 
        y_train[train], 
        batch_size = batch_size
    ),
    epochs=epochs,
    validation_data = (X_train[val], y_train[val]),
    verbose=1,
    steps_per_epoch = X_train[train].shape[0] // batch_size,
    callbacks = [learning_rate_reduction]
)

In [ ]:
tf.config.run_functions_eagerly(True)

# Fine Tuning
base_modelQ4.trainable = True
new_optimizer = Adam(learning_rate=0.0001, beta_1=0.9, beta_2=0.999, decay=0.0, amsgrad=False)
modelQ4.compile(
    loss = loss_function,
    metrics = ["accuracy"],
    optimizer = new_optimizer
)

epochs = 100
callbacks_list = [learning_rate_reduction, model_checkpoint, log_csv, early_stopping]
modelQ4.fit(
    train_datagen.flow(
        X_train[train],
        y_train[train],
        batch_size = batch_size
    ),
    epochs = epochs,
    validation_data = (X_train[val], y_train[val]),
    verbose = 1,
    steps_per_epoch = X_train[train].shape[0] // batch_size,
    callbacks = callbacks_list
)

#modelQ4.save(f'data_models/224x224_bs8_300fine_{model_no}.h5', overwrite=True)
# serialize model to JSON
classifier_json = modelQ4.to_json()
with open(f"../models/models_jsons/MODEL_NAME_{model_no}.json", "w") as json_file:
    json_file.write(classifier_json)
# serialize weights to HDF5
modelQ4.save_weights(f"../models/model_weights/MODEL_NAME_{model_no}.weights.h5")
print("Saved weights to disk")

In [ ]:
# K-fold Cross Validation model evaluation
batch_size = 4

for (train, val), model_no in zip((kfold.split(X_train, y_train)), range(1,6)):

    base_model = keras.applications.Xception(
        weights='imagenet',  # Load weights pre-trained on ImageNet.
        input_shape=input_shape,
        include_top=False)
   
    ## callbacks part
    mc_path = f"../models/best_models/{base_model.name}_kfold-model{model_no}.weights.h5"
   
    model_checkpoint = ModelCheckpoint(filepath = mc_path,
                                   monitor = 'val_accuracy',
                                   mode = 'max',
                                   verbose = 1,
                                   save_best_only = True,
                                   save_weights_only = True)
   
   
    learning_rate_reduction = ReduceLROnPlateau(
        monitor='val_accuracy',
        patience=2,
        verbose=1,
        factor=0.1,
        min_lr=0.00001
    )
   
    log_csv = CSVLogger(f'../models/model_logs/{base_model.name}_model{model_no}_logs.csv', separator = ',', append = False)


    early_stopping = EarlyStopping(monitor='loss', patience = 10)
    
    callbacks_list = [learning_rate_reduction, model_checkpoint, log_csv]
   
   
    base_modelQ4 = base_model
   
    base_modelQ4.trainable = False
    inputs = keras.Input(shape=input_shape)
    
   
    q4 = base_modelQ4(inputs, training=False)
    q4 = keras.layers.Conv2D(64, (1,1), activation = 'relu', padding = 'same', name = "pikachu")(q4)
    q4 = keras.layers.GlobalAveragePooling2D()(q4)
    q4 = keras.layers.Dropout(0.2)(q4)
    q4 = keras.layers.Flatten()(q4)
    q4 = keras.layers.Dense(256, activation='relu')(q4)
    q4 = keras.layers.Dropout(0.2)(q4)
    q4 = keras.layers.Dense(128, activation='relu')(q4)
    q4 = keras.layers.Dropout(0.3)(q4)
    q4 = keras.layers.Dense(64, activation='relu')(q4)
    q4 = keras.layers.Dropout(0.3)(q4)
    
    outputs = keras.layers.Dense(num_classes, activation='softmax')(q4)
    modelQ4 = keras.Model(inputs, outputs)
   
   
    modelQ4.compile(loss= loss_function,
                    metrics=["accuracy"],
                    optimizer=optimizer)

    print(y_train.shape)
   
    epochs = 50
    modelQ4.fit(train_datagen.flow(X_train[train], y_train[train], batch_size = batch_size),
                          epochs=epochs,
                          validation_data = (X_train[val], y_train[val]),
                          verbose=1,
                          steps_per_epoch = X_train[train].shape[0] // batch_size,
                          callbacks = [learning_rate_reduction])
   
    # Fine Tuning
    base_modelQ4.trainable = True
    modelQ4.compile(loss= loss_function,
                    metrics=["accuracy"],
                    optimizer = optimizer)
   
   
    epochs = 100
    callbacks_list = [learning_rate_reduction, model_checkpoint, log_csv,early_stopping]
    modelQ4.fit(train_datagen.flow(X_train[train], y_train[train], batch_size = batch_size),
                          epochs = epochs,
                          validation_data = (X_train[val], y_train[val]),
                          verbose=1,
                          steps_per_epoch= X_train[train].shape[0] // batch_size,
                          callbacks = callbacks_list)
    
    #modelQ4.save(f'data_models/224x224_bs8_300fine_{model_no}.h5', overwrite=True)
    # serialize model to JSON
    classifier_json = modelQ4.to_json()
    with open(f"../models/models_jsons/MODEL_NAME_{model_no}.json", "w") as json_file:
        json_file.write(classifier_json)
    # serialize weights to HDF5
    modelQ4.save_weights(f"../models/model_weights/MODEL_NAME_{model_no}.h5")
    print("Saved weights to disk")
   
      # Generate a print
    print('------------------------------------------------------------------------')
    print(f'Training for fold {model_no} ...')
     
    json_file = open(f'../models/models_jsons/MODEL_NAME_{model_no}.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    # load weights into new model
    loaded_model.load_weights(f"../models/best_models/{base_model.name}_kfold-model{model_no}.hdf5")
    print("Loaded model from disk")
    
    loaded_model.compile(loss= loss_function,
                    metrics=["accuracy"],
                    optimizer = optimizer)
   
    
    # Generate generalization metrics
    scores = loaded_model.evaluate(X_train[val], y_train[val], verbose = 0)
    print(f'Score for fold {model_no}: {loaded_model.metrics_names[0]} of {scores[0]}; {loaded_model.metrics_names[1]} of {scores[1]*100}%\n\n')
    val_acc_per_fold.append(scores[1] * 100)
    val_loss_per_fold.append(scores[0])
   
       
    test_loss_eval, test_accuracy_eval = loaded_model.evaluate(X_test, y_test, verbose = 0)
    print("Eval: accuracy = %f  ;  loss = %f" % (test_accuracy_eval, test_loss_eval))
           
    test_acc_per_fold_eva.append(test_accuracy_eval)
    test_loss_per_fold_eva.append(test_loss_eval)    


    Y_pred = loaded_model.predict(X_test)
    Y_pred_classes = np.argmax(Y_pred, axis=1)
    Y_true = np.argmax(y_test, axis=1)
    confusion_mtx = confusion_matrix(Y_true, Y_pred_classes)
    plot_confusion_matrix(confusion_mtx, classes=classes, model_no = model_no)  
   
    test_accuracy_pred = accuracy_score(Y_true, Y_pred_classes)
   
    print('\nAccuracy: {:.2f}\n'.format(accuracy_score(Y_true, Y_pred_classes)))
   
    print('Micro Precision: {:.2f}'.format(precision_score(Y_true, Y_pred_classes, average='micro')))
    print('Micro Recall: {:.2f}'.format(recall_score(Y_true, Y_pred_classes, average='micro')))
    print('Micro F1-score: {:.2f}\n'.format(f1_score(Y_true, Y_pred_classes, average='micro')))
   
    print('Macro Precision: {:.2f}'.format(precision_score(Y_true, Y_pred_classes, average='macro')))
    print('Macro Recall: {:.2f}'.format(recall_score(Y_true, Y_pred_classes, average='macro')))
    print('Macro F1-score: {:.2f}\n'.format(f1_score(Y_true, Y_pred_classes, average='macro')))
   
    print('Weighted Precision: {:.2f}'.format(precision_score(Y_true, Y_pred_classes, average='weighted')))
    print('Weighted Recall: {:.2f}'.format(recall_score(Y_true, Y_pred_classes, average='weighted')))
    print('Weighted F1-score: {:.2f}'.format(f1_score(Y_true, Y_pred_classes, average='weighted')))
   
    test_acc_per_fold_pred.append(test_accuracy_pred)    
   
    print('\nClassification Report\n')
    print(classification_report(Y_true, Y_pred_classes, target_names= classes, digits=4))


In [ ]:
# == Provide average scores ==
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(test_acc_per_fold_pred)):
  print('------------------------------------------------------------------------')
  print(f'> Fold {i+1}  - Accuracy: {test_acc_per_fold_pred[i]}%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(test_acc_per_fold_pred)} (+- {np.std(test_acc_per_fold_pred)})')
#print(f'> Loss: {np.mean(loss_per_fold)}')
print('------------------------------------------------------------------------')

#%% 1 genereal model

base_model = keras.applications.Xception(  # Xception, VGG16, MobileNetV2,ResNet50V2,ResNet101V2 //InceptionResNetV2 299 ,EfficientNetV2L
       weights='imagenet',  # Load weights pre-trained on ImageNet.
       input_shape=input_shape,
       include_top=False)

   ## callbacks part
mc_path = f"../models/best_models/{base_model.name}.hdf5"

model_checkpoint = ModelCheckpoint(filepath=mc_path,
                                      monitor='val_accuracy',
                                      mode='max',
                                      verbose=1,
                                      save_best_only=True,
                                      save_weights_only=True)

learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy',
                                               patience=2,
                                               verbose=1,
                                               factor=0.1,
                                               min_lr=0.00001)

log_csv = CSVLogger(
       f'../models/model_logs/{base_model.name}_logs.csv', separator=',', append=False)

early_stopping = EarlyStopping(monitor='loss', patience=10)

callbacks_list = [learning_rate_reduction, model_checkpoint, log_csv]

base_modelQ4 = base_model

base_modelQ4.trainable = False
inputs = keras.Input(shape=input_shape)

q4 = base_modelQ4(inputs, training=False)
q4 = keras.layers.Conv2D(
       64, (1, 1), activation='relu', padding='same', name="pikachu")(q4)
q4 = keras.layers.GlobalAveragePooling2D()(q4)
q4 = keras.layers.Dropout(0.2)(q4)
q4 = keras.layers.Flatten()(q4)
q4 = keras.layers.Dense(256, activation='relu')(q4)
q4 = keras.layers.Dropout(0.2)(q4)
q4 = keras.layers.Dense(128, activation='relu')(q4)
q4 = keras.layers.Dropout(0.3)(q4)
q4 = keras.layers.Dense(64, activation='relu')(q4)
q4 = keras.layers.Dropout(0.3)(q4)

outputs = keras.layers.Dense(num_classes, activation='softmax')(q4)
modelQ4 = keras.Model(inputs, outputs)

modelQ4.compile(loss=loss_function,
                   metrics=["accuracy"],
                   optimizer=optimizer)

epochs = 50
modelQ4.fit_generator(train_datagen.flow(X_train, y_train, batch_size=batch_size),
                         epochs=epochs,
                         validation_data=(X_val, y_val),
                         verbose=1,
                         steps_per_epoch=X_train.shape[0] // batch_size,
                         callbacks=[learning_rate_reduction])

   # Fine Tuning
base_modelQ4.trainable = True
modelQ4.compile(loss=loss_function,
                   metrics=["accuracy"],
                   optimizer=optimizer)

epochs = 100
callbacks_list = [learning_rate_reduction,
                     model_checkpoint, log_csv, early_stopping]
modelQ4.fit_generator(train_datagen.flow(X_train, y_train, batch_size=batch_size),
                         epochs=epochs,
                         validation_data=(X_val, y_val),
                         verbose=1,
                         steps_per_epoch=X_train.shape[0] // batch_size,
                         callbacks=callbacks_list)

   #modelQ4.save(f'data_models/224x224_bs8_300fine_{model_no}.h5', overwrite=True)
   # serialize model to JSON
classifier_json = modelQ4.to_json()
with open(f"../models/models_jsons/224x224_bs8_300fine.json", "w") as json_file:
        json_file.write(classifier_json)
    # serialize weights to HDF5
modelQ4.save_weights(
        f"../models/model_weights/224x224_bs8_300fine.h5")
print("Saved weights to disk")

    # Generate a print
print('------------------------------------------------------------------------')
print(f'Training ')




In [ ]:
json_path = "../models/models_jsons/JSON_NAME.json"
model_path = "../models/best_models/MODEL_NAME.weights.h5"

json_file = open(json_path, 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

# load weights into new model
loaded_model.load_weights(model_path)
print("Loaded model from disk")

loaded_model.compile(loss=loss_function,
                         metrics=["accuracy"],
                         optimizer=optimizer)

# Generate generalization metrics
scores = loaded_model.evaluate(X_val, y_val, verbose=0)
print(f'Score: {loaded_model.metrics_names[0]} of {scores[0]}; {loaded_model.metrics_names[1]} of {scores[1]*100}%\n\n')


test_loss_eval, test_accuracy_eval = loaded_model.evaluate(
        X_test, y_test, verbose=0)
print("Eval: accuracy = %f  ;  loss = %f" %
          (test_accuracy_eval, test_loss_eval))


Y_pred = loaded_model.predict(X_test)
Y_pred_classes = np.argmax(Y_pred, axis=1)
Y_true = np.argmax(y_test, axis=1)
confusion_mtx = confusion_matrix(Y_true, Y_pred_classes)
plot_confusion_matrix(confusion_mtx, classes=classes)

test_accuracy_pred = accuracy_score(Y_true, Y_pred_classes)

print('\nAccuracy: {:.2f}\n'.format(
        accuracy_score(Y_true, Y_pred_classes)))

print('Micro Precision: {:.2f}'.format(
        precision_score(Y_true, Y_pred_classes, average='micro')))
print('Micro Recall: {:.2f}'.format(
        recall_score(Y_true, Y_pred_classes, average='micro')))
print('Micro F1-score: {:.2f}\n'.format(f1_score(Y_true,
          Y_pred_classes, average='micro')))

print('Macro Precision: {:.2f}'.format(
        precision_score(Y_true, Y_pred_classes, average='macro')))
print('Macro Recall: {:.2f}'.format(
        recall_score(Y_true, Y_pred_classes, average='macro')))
print('Macro F1-score: {:.2f}\n'.format(f1_score(Y_true,
          Y_pred_classes, average='macro')))

print('Weighted Precision: {:.2f}'.format(
        precision_score(Y_true, Y_pred_classes, average='weighted')))
print('Weighted Recall: {:.2f}'.format(
        recall_score(Y_true, Y_pred_classes, average='weighted')))
print('Weighted F1-score: {:.2f}'.format(f1_score(Y_true,
          Y_pred_classes, average='weighted')))

print('\nClassification Report\n')
print(classification_report(
    Y_true,
    Y_pred_classes,
    target_names=classes,
    digits=4
))


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import model_from_json
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt

def enable_mc_dropout(model):
    """Modifies the model to enable Monte Carlo dropout at inference time."""
    for layer in model.layers:
        if isinstance(layer, tf.keras.layers.Dropout):
            layer.training = True
    return model

def mc_dropout_predictions(model, X, n_simulations=50):
    """Runs multiple forward passes with MC dropout to estimate uncertainty."""
    preds = np.array([model.predict(X, verbose=0) for _ in range(n_simulations)])

    for prd in preds:
        print(prd)
    
    mean_pred = preds.mean(axis=0)  # Mean prediction
    std_pred = preds.std(axis=0)    # Standard deviation (uncertainty measure)
    return mean_pred, std_pred

# Load the model architecture
json_path = "../models/models_jsons/JSON_NAME.json"
model_path = "../models/best_models/MODEL_NAME.weights.h5"

with open(json_path, 'r') as json_file:
    loaded_model_json = json_file.read()
loaded_model = model_from_json(loaded_model_json)
loaded_model.load_weights(model_path)
print("Loaded model from disk")

# Compile the model
loaded_model.compile(loss=loss_function, metrics=["accuracy"], optimizer=optimizer)

# Evaluate standard performance
scores = loaded_model.evaluate(X_val, y_val, verbose=0)
print(f'Score: {loaded_model.metrics_names[0]} of {scores[0]}; {loaded_model.metrics_names[1]} of {scores[1]*100}%')

test_loss_eval, test_accuracy_eval = loaded_model.evaluate(X_test, y_test, verbose=0)
print(f"Eval: accuracy = {test_accuracy_eval:.6f}  ;  loss = {test_loss_eval:.6f}")

# Enable MC Dropout
mc_model = enable_mc_dropout(loaded_model)

# Monte Carlo Dropout Predictions
Y_pred_mean, Y_pred_std = mc_dropout_predictions(mc_model, X_test, n_simulations=100)
Y_pred_classes = np.argmax(Y_pred_mean, axis=1)
Y_true = np.argmax(y_test, axis=1)

# Compute Uncertainty (Standard deviation per sample)
uncertainty = np.mean(Y_pred_std, axis=1)

# Confusion Matrix
confusion_mtx = confusion_matrix(Y_true, Y_pred_classes)
plot_confusion_matrix(confusion_mtx, classes=classes)

test_accuracy_pred = accuracy_score(Y_true, Y_pred_classes)
print(f'\nAccuracy: {test_accuracy_pred:.2f}\n')

print('Micro Precision: {:.2f}'.format(precision_score(Y_true, Y_pred_classes, average='micro')))
print('Micro Recall: {:.2f}'.format(recall_score(Y_true, Y_pred_classes, average='micro')))
print('Micro F1-score: {:.2f}\n'.format(f1_score(Y_true, Y_pred_classes, average='micro')))

print('Macro Precision: {:.2f}'.format(precision_score(Y_true, Y_pred_classes, average='macro')))
print('Macro Recall: {:.2f}'.format(recall_score(Y_true, Y_pred_classes, average='macro')))
print('Macro F1-score: {:.2f}\n'.format(f1_score(Y_true, Y_pred_classes, average='macro')))

print('Weighted Precision: {:.2f}'.format(precision_score(Y_true, Y_pred_classes, average='weighted')))
print('Weighted Recall: {:.2f}'.format(recall_score(Y_true, Y_pred_classes, average='weighted')))
print('Weighted F1-score: {:.2f}\n'.format(f1_score(Y_true, Y_pred_classes, average='weighted')))

# Classification Report
print('\nClassification Report\n')
print(classification_report(Y_true, Y_pred_classes, target_names=classes, digits=4))